In [1]:
import pandas as pd
import numpy as np
import string
from nltk.stem import WordNetLemmatizer
from numpy import math

In [2]:
# Read in ingredients
ing_df = pd.read_csv("ingredients.csv").set_index('ingredient')

In [3]:
def format_ingredient(s):
    lm = WordNetLemmatizer()
    printable = string.printable
    PERMITTED_CHARS = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ " 

    filter(lambda x: x in printable, s)
    s = "".join(c for c in s if c in PERMITTED_CHARS)
    s = lm.lemmatize(s.lower())

    return s    

# Given an ingredient, return all subsets of the ingredient string as a list of strings
def get_substrings(ingredient):
    # Base case - Return an empty list of the string is empty
    if ingredient == '':
        return []
    
    word_list = ingredient.split(' ')
    substring_list = []
    substring_list.append(ingredient)
    
    # Base case - Return a list with only the ingredient if there is only one word
    if len(word_list) == 1:
        return substring_list
    
    # Recursive Case
    for i in range(len(word_list)):
        new_list = np.array(word_list, copy=True).tolist()
        new_list.remove(word_list[i])
        substring_word = ' '.join(new_list)
        substring_list.append(substring_word)
        
        sub_list = get_substrings(substring_word)
        for w in sub_list:
            substring_list.append(w)

    return substring_list

# Dedupes the output list of get_substrings function
def get_substrings_deduped(substring_list):
    return np.unique(get_substrings(substring_list)).tolist()

def broadmatch(ing):
    if len(ing.split(' ')) > 10:
        return ing
    
    df = pd.DataFrame(get_substrings_deduped(ing), columns=['ingredient']).set_index('ingredient')
    joined_df = df.join(ing_df, lsuffix = 'l', rsuffix='r')
    normalized_ingredient = joined_df['num_recipes'].idxmax()
    return normalized_ingredient if not type(normalized_ingredient) == float else ing

In [7]:
norm_ing_arr = []
for i in range(len(ing_df.index)):
    norm_ing_arr.append([format_ingredient(ing_df.index[i]), broadmatch(format_ingredient(ing_df.index[i]))])

In [8]:
norm_ing_df = pd.DataFrame(norm_ing_arr, columns = ['Original Ingredient', 'Broadmatched Ingredient'])
norm_ing_df.head()

norm_ing_df.to_csv("broadmatched_ingredients.csv", index=False)